In [1]:
import csv
import random
import unidecode
import warnings

import pandas as pd
import numpy as np

import hu_core_ud_lg

import nltk
from nltk.corpus import stopwords

from elasticsearch import Elasticsearch, helpers
from elasticsearch_dsl import Search, Q
from elasticsearch.helpers import bulk
from elasticsearch.exceptions import ElasticsearchWarning

from tqdm.notebook import tqdm

C:\Users\User\anaconda3\lib\site-packages\spacy\util.py:275: UserWarning: [W031] Model 'hu_core_ud_lg' (0.3.1) requires spaCy v2.1 and is incompatible with the current spaCy version (2.3.7). This may lead to unexpected results or runtime errors. To resolve this, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


In [2]:
es = Elasticsearch("http://localhost:9200")

In [3]:
faq_csv = ".data/faq_with_splits_tokenized.csv"

In [4]:
df = pd.read_csv(faq_csv)

In [5]:
df.head()

,short_question,long_question,answer,main_category,sub_category,split
0,Vállsérülés gyógytorna,Vállsérülés gyógytorna Az lenne a kérdésem hog...,"edzeni kénealapjába véve mindent váll, hát, tr...",Egészség,"Sérülések, balesetek",train
1,Heti hányszor dehidtratáljam a szervezetemet c...,Heti hányszor dehidtratáljam a szervezetemet c...,"Nem értem miért akarod magad dehidratálni, nem...",Egészség,Bio életmód,train
2,A Doctor Who ban ki a vörös hajú lány,A Doctor Who ban ki a vörös hajú lány,Vagy Amy Pond vagy Donna Noble,Szórakozás,"Filmek, sorozatok",train
3,Vettem 2 törpenyuszit a szobamba 4 5 honapja d...,Vettem 2 törpenyuszit a szobamba 4 5 honapja d...,Egy törpenyuszihoz nagyon sok türelem kell. Pr...,Állatok,Kisemlősök,train
4,Fityma szűkületem van,Fityma szűkületem van 15 éves fiú vagyok és mi...,Valószínű. Esetleg még a fitymaféked lehet röv...,Egészség,Férfiak egészsége,train


In [6]:
warnings.simplefilter('ignore', ElasticsearchWarning)

In [7]:
def delete_indices():
    indices = es.indices.get('questions*')

    for index in indices:
        print(f"Deleting index '{index}'")
        es.indices.delete(index=index, ignore=[404])

In [8]:
delete_indices()

Deleting index 'questions_allatok'
Deleting index 'questions_egeszseg'
Deleting index 'questions_szamitastechnika'
Deleting index 'questions_szorakozas'


In [9]:
def generate_data():
    
    for index, row in df.iterrows():
        yield {
            "_index": "questions_" + unidecode.unidecode(row.main_category.lower()),
            "short_question": row.short_question,
            "long_question": row.long_question,
            "category": row.main_category,
            "answer": row.answer
        }

In [10]:
_ = bulk(es, generate_data())

In [11]:
def generate_query(question):
    return {
        "query": {
            "match": {
                "long_question": {
                    "query": question,
                }
            }
        }
    }
    
def find_answer(question, index="questions*"):
    body = generate_query(question)
    res = es.search(index=index, body=body)
    
    return res['hits']['hits'][0]['_source']

In [12]:
question = input()

result = find_answer(question)

print(f"Short question:\n{result['short_question']}")
print()
print(f"Long question:\n{result['long_question']}")
print()
print(f"Answer:\n{result['answer']}")

Miért ilyen nehéz megtartani az életkedvem a BME-n?
Short question:
Miért van a BME GPK n igény olyan jellegű alpári rendezvényekre mint pl a Slip

Long question:
Miért van a BME GPK n igény olyan jellegű alpári rendezvényekre mint pl a Slip

Answer:
Két tippem van: vagy kármános vagy és idegesít a felhajtás, vagy simán savanyú a szőlő. Más egyebet nem tudok elképzelni, miért írtad ki ezt a kérdést. A választ ugyanis nyilván te is tudod: népes egyetemi kar rengeteg fiatallal, akik között elég sok bulizni vágyót találni, a te szemedben alpári igényekkel.


-------------------------------------------------------------------------------------
-------------------------------------------------------------------------------------

In [13]:
index_file = "index.json"
batch_size = 1000

In [14]:
df.head()

,short_question,long_question,answer,main_category,sub_category,split
0,Vállsérülés gyógytorna,Vállsérülés gyógytorna Az lenne a kérdésem hog...,"edzeni kénealapjába véve mindent váll, hát, tr...",Egészség,"Sérülések, balesetek",train
1,Heti hányszor dehidtratáljam a szervezetemet c...,Heti hányszor dehidtratáljam a szervezetemet c...,"Nem értem miért akarod magad dehidratálni, nem...",Egészség,Bio életmód,train
2,A Doctor Who ban ki a vörös hajú lány,A Doctor Who ban ki a vörös hajú lány,Vagy Amy Pond vagy Donna Noble,Szórakozás,"Filmek, sorozatok",train
3,Vettem 2 törpenyuszit a szobamba 4 5 honapja d...,Vettem 2 törpenyuszit a szobamba 4 5 honapja d...,Egy törpenyuszihoz nagyon sok türelem kell. Pr...,Állatok,Kisemlősök,train
4,Fityma szűkületem van,Fityma szűkületem van 15 éves fiú vagyok és mi...,Valószínű. Esetleg még a fitymaféked lehet röv...,Egészség,Férfiak egészsége,train


In [15]:
nlp = hu_core_ud_lg.load()

C:\Users\User\anaconda3\lib\site-packages\spacy\util.py:275: UserWarning: [W031] Model 'hu_core_ud_lg' (0.3.1) requires spaCy v2.1 and is incompatible with the current spaCy version (2.3.7). This may lead to unexpected results or runtime errors. To resolve this, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


In [16]:
nltk.download('stopwords', quiet=True)
stop_words = set(stopwords.words('hungarian'))

list(stop_words)[:5]

['alatt', 'ki', 'szinte', 'át', 'vissza']

In [17]:
def filter_stopwords(text):
    return " ". join([w for w in text if not w in stop_words])

In [18]:
def embed(sentence):
    docs = nlp(sentence)  
    avg = sum([doc.vector for doc in docs]) / len(docs)
    return avg

In [19]:
def embed_docs(docs):
    return [embed(filter_stopwords(doc)) for doc in docs]

In [20]:
mc = df.main_category.unique().tolist()
main_categories = [unidecode.unidecode(c.lower()) for c in mc] 
main_categories

['egeszseg', 'szorakozas', 'allatok', 'szamitastechnika']

In [21]:
delete_indices()

Deleting index 'questions_allatok'
Deleting index 'questions_egeszseg'
Deleting index 'questions_szamitastechnika'
Deleting index 'questions_szorakozas'


In [22]:
with open(index_file) as index:
    source = index.read().strip()
    
    for category in main_categories:
        index_name = "questions_" + category
        print(f"Creating index '{index_name}'")
        es.indices.create(index=index_name, body=source)

Creating index 'questions_egeszseg'
Creating index 'questions_szorakozas'
Creating index 'questions_allatok'
Creating index 'questions_szamitastechnika'


In [23]:
def index_batch(docs):
    short_questions = [doc["short_question"] for doc in docs]
    short_questions_vectors = embed_docs(short_questions)

    index_names = ["questions_" + unidecode.unidecode(doc["main_category"].lower()) for doc in docs]

    requests = []
    
    for i, doc, in enumerate(docs):
        request = doc
        request["_op_type"] = "index"
        request["_index"] = index_names[i]
        request["short_question_vector"] = short_questions_vectors[i]
        requests.append(request)
    helpers.bulk(es, requests)

In [24]:
def index_data(df):
    num_of_batches = 10 #df.shape[0] // batch_size
    
    if df.shape[0] % batch_size != 0:
        num_of_batches += 1

    progress_bar = tqdm(desc=f"Indexing batches", total=num_of_batches, leave=True)
        
    for i in range(num_of_batches):
        batch = df[i * batch_size : (i + 1) * batch_size]
        batch_as_dict = batch.apply(lambda row: row.to_dict(), axis = 1).tolist()

        index_batch(batch_as_dict)
        
        progress_bar.update()
        progress_bar.refresh()

In [25]:
index_data(df)

Indexing batches:   0%|          | 0/11 [00:00<?, ?it/s]

In [26]:
script_query = {
    "script_score": {
        "query": {"match_all": {}},
        "script": {
            "source": "cosineSimilarity(params.query_vector, 'short_question_vector') + 1.0",
            "params": {"query_vector": embed(filter_stopwords(question))}
        }
    }
}


response = es.search(
    index='questions*',
    body={
        "size": 10,
        "query": script_query
    }
)

In [27]:
for hit in response["hits"]["hits"]:
    print("id: {}, score: {}".format(hit["_id"], hit["_score"]))
    print(hit["_source"]["short_question"])
    print(hit["_source"]["answer"])
    print()

id: uRSUFIABYO0TvOV9zBaK, score: 1.9856137
Újra szeretném telepíteni a gépem és a boot menüben nem érzékeli a pendrive ot Ez miért van Mit lehet vele csinálni
„Laptopom van”Kár volt vesződnöd ezzel a túl precíz típusmeghatározással, elég lett volna annyi, hogy „fekete”.

id: HRSUFIABYO0TvOV9awuP, score: 1.9854136
Milyen játékokat vinne el és mit érdemes rajta fejleszteni GTA V esetleg
Kb a gt730 viszi full lowon 25-30fps-el.Ez meg ugy sem... Vegyel jobb procit meg egy gtx 750-et ha jatszani akarsz a Gta-val.Gt amugy sem jatekra valo.

id: UhSUFIABYO0TvOV9agrT, score: 1.9852462
Szent Johanna Gimi Fanok A második rész Együtt 2 felébe mi történt
1. Elmész könyvtárba és kiveszed2. Megveszed.Választhatsz. De azt ne várd, hogy majd valaki elkezdi begépelni...

id: DBSUFIABYO0TvOV9agrT, score: 1.984168
Léteznek még ilyen tini magazinok mint pl Az IM es a bravo
Kedves kérdést kiíró leányzó!Az én veled egykorú lányom a Joy-t, A Bravo Girl-t és az Instyle-t lapozgatja naphosszatt!Remélem még akt